In [ ]:
!pip install textblob #language recognition

In [4]:
import requests
import csv
import pandas as pd
import numpy as np
import datetime #today's date
from textblob import TextBlob 
from bs4 import BeautifulSoup
import re
import time #delay for scrapping multiple articles

In [5]:
url = "https://www.theinnovation.eu/comment-tuer-linnovation-avec-lanalyse-financiere/45"

In [6]:
df= pd.DataFrame(columns=['art_content','art_content_html','art_extract_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag']  )
df

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag


In [7]:
""" 
In: url
Out: a tuple of strings
"""

def get_all(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    """['art_content','art_content_html','art_extract_datetime','art_lang',
    'art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag']  )
    """

    art_content_html = soup.find("div", {"class": "entry-content"})
    art_content = art_content_html.text.replace('\xa0', '')
    art_extract_datetime = soup.find(
        "meta", {"property": "article:modified_time"})["content"]
    art_lang = soup.find("meta", {"property": "og:locale"})["content"]
    art_title = soup.find("meta", {"property": "og:title"})["content"]
    art_url = soup.find("meta", {"property": "og:url"})["content"]
    src_name = soup.find("a", {"rel": "home"}).text
    src_type = "xpath_source" #default value
    src_url = soup.find("a", {"rel": "home"})["href"]
    src_img = soup.find("meta", {"property": "og:image"})["content"]
    art_auth = soup.find("a", {"rel": "author"}).text
    art_tags = soup.find_all("a", {"rel": "tag"})
    art_tag = []
    for tag in art_tags:
        art_tag.append(str(tag.text))
    return art_content, art_content_html, art_extract_datetime, art_lang, art_title, art_url, \
        src_name, src_type, src_url, src_img, art_auth, art_tag, 

'art_content' [0],'art_content_html'[1],'art_extract_datetime'[2],'art_lang'[3],
    'art_title'[4],'art_url'[5],'src_name'[6],'src_type'[7],'src_url'[8],'src_img'[9],'art_auth'[10],'art_tag'[11]
    

In [9]:
get_all(url)[11] #example with the article tags

['Fiche de lecture',
 'Innovation',
 'analyse financière',
 'cash flow',
 'discovery-driven-planning',
 'Innovation']

In [14]:
df.loc[1] = get_all(url) #to fill in the df

In [16]:
df

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
1,\nFiche de lecture\nInnovation killers : how f...,"[\n, [[<strong>Fiche de lecture</strong>]], \n...",2010-05-14T15:55:01+00:00,fr_FR,Comment tuer l’innovation avec l’analyse finan...,https://www.theinnovation.eu/comment-tuer-linn...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Fiche de lecture, Innovation, analyse financi..."


In [69]:
"""
In: url
Out: url
the function will return the url of the previous article
"""

def get_prev(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    prev_soup = soup.find("a", {"rel": "prev"})
    if prev_soup is not None:
        prev_url = prev_soup["href"]
        return prev_url
    else:
        return None
    
"""
In: url
Out: url
the function will return the url of the following article
"""
    
def get_next(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    next_soup = soup.find("a", {"rel": "next"})
    if next_soup is not None:
        next_url = next_soup["href"]
        return next_url
    else:
        return None


In [70]:
url = "https://www.theinnovation.eu/comment-tuer-linnovation-avec-lanalyse-financiere/45"

In [71]:
get_next(url)

'https://www.theinnovation.eu/leffondrement-du-sens-dans-les-organisations/50'

In [65]:
"""
In: url
Out: url
Finds the first article of the website
"""

def find_first(url):
    while get_prev(url) is not None:
        url = get_prev(url)
        print(url)
    return url

"""
In: url
Out: url
Finds the last article of the website
"""

def find_last(url):
    while get_next(url) is not None:
        url = get_next(url)
        print(url)
    return url

In [77]:
url = find_first(url)
url

https://www.theinnovation.eu/fichier-edwige-innovation-incrementale-innovation-radicale-ou-innovation-de-rupture/44
https://www.theinnovation.eu/linnovation-avec-les-lead-user/42
https://www.theinnovation.eu/comment-eviter-les-catastrophes-ecologiques/41
https://www.theinnovation.eu/construction-identitaire-dans-les-mondes-virtuels/39
https://www.theinnovation.eu/innover-sur-le-business-model/36
https://www.theinnovation.eu/la-generation-didees-dutilisateurs-de-services-bases-sur-de-la-technologie/33
https://www.theinnovation.eu/les-business-model-de-lopen-source/29
https://www.theinnovation.eu/management-des-idees-et-open-innovation/28
https://www.theinnovation.eu/appel-a-projet-oseo-cnc-une-tendance-au-developpement-plutot-qu%e2%80%99a-la-recherche/26
https://www.theinnovation.eu/les-licences-du-web-20/20
https://www.theinnovation.eu/appel-a-projet-oseo-cnc-pour-linnovation-en-audio-visuel-et-multimedia/14
https://www.theinnovation.eu/innovation-et-changement-institutionnel/11
https:

'https://www.theinnovation.eu/le-forum-innovation-et-tourisme/5'

In [75]:
"""
In: url of the 1st article
Out: list of urls
Finds all the following articles of the website
"""

def all_articles(url):
    all_art = []
    all_art.append(url)
    while get_next(url) is not None:
        url = get_next(url)
        print(url)
        all_art.append(url)
    return all_art

In [78]:
url

'https://www.theinnovation.eu/le-forum-innovation-et-tourisme/5'

In [79]:
liste_articles = all_articles(url)

https://www.theinnovation.eu/innovation-et-changement-institutionnel/11
https://www.theinnovation.eu/appel-a-projet-oseo-cnc-pour-linnovation-en-audio-visuel-et-multimedia/14
https://www.theinnovation.eu/les-licences-du-web-20/20
https://www.theinnovation.eu/appel-a-projet-oseo-cnc-une-tendance-au-developpement-plutot-qu%e2%80%99a-la-recherche/26
https://www.theinnovation.eu/management-des-idees-et-open-innovation/28
https://www.theinnovation.eu/les-business-model-de-lopen-source/29
https://www.theinnovation.eu/la-generation-didees-dutilisateurs-de-services-bases-sur-de-la-technologie/33
https://www.theinnovation.eu/innover-sur-le-business-model/36
https://www.theinnovation.eu/construction-identitaire-dans-les-mondes-virtuels/39
https://www.theinnovation.eu/comment-eviter-les-catastrophes-ecologiques/41
https://www.theinnovation.eu/linnovation-avec-les-lead-user/42
https://www.theinnovation.eu/fichier-edwige-innovation-incrementale-innovation-radicale-ou-innovation-de-rupture/44
https:

In [80]:
liste_articles

['https://www.theinnovation.eu/le-forum-innovation-et-tourisme/5',
 'https://www.theinnovation.eu/innovation-et-changement-institutionnel/11',
 'https://www.theinnovation.eu/appel-a-projet-oseo-cnc-pour-linnovation-en-audio-visuel-et-multimedia/14',
 'https://www.theinnovation.eu/les-licences-du-web-20/20',
 'https://www.theinnovation.eu/appel-a-projet-oseo-cnc-une-tendance-au-developpement-plutot-qu%e2%80%99a-la-recherche/26',
 'https://www.theinnovation.eu/management-des-idees-et-open-innovation/28',
 'https://www.theinnovation.eu/les-business-model-de-lopen-source/29',
 'https://www.theinnovation.eu/la-generation-didees-dutilisateurs-de-services-bases-sur-de-la-technologie/33',
 'https://www.theinnovation.eu/innover-sur-le-business-model/36',
 'https://www.theinnovation.eu/construction-identitaire-dans-les-mondes-virtuels/39',
 'https://www.theinnovation.eu/comment-eviter-les-catastrophes-ecologiques/41',
 'https://www.theinnovation.eu/linnovation-avec-les-lead-user/42',
 'https://w

In [90]:
"""
In: list of urls
Out: dataframe with all scraped articles
"""
def scrap_all(list_urls):
    i = 1
    df= pd.DataFrame(columns=['art_content','art_content_html','art_extract_datetime',\
                              'art_lang','art_title','art_url','src_name','src_type','src_url',\
                              'src_img','art_auth','art_tag'])
    for url in list_urls:
        df.loc[i] = get_all(url)
        i+=1
    return df
        
    
    

In [91]:
"""
In: url of a random article
Out: dataframe with all scraped articles
"""

def do_it_all(url):
    url = find_first(url)
    list_urls = all_articles(url)
    scrap_all(list_urls)

In [92]:
scrap_all(liste_articles)

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
1,\n\nOn ne parle pas souvent d’innovation dans ...,"[\n, [[<img alt="""" src=""http://www.theinnovati...",2010-05-14T16:05:07+00:00,fr_FR,29-30 novembre – Forum Innovation et Tourisme,https://www.theinnovation.eu/le-forum-innovati...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Agenda, Innovation, Second Life, Tourisme]"
2,\n Fiche de lecture\nInstitutional Change in ...,"[\n, [[<strong> Fiche de lecture</strong>]], ...",2017-10-30T10:15:01+00:00,fr_FR,Innovation et changement institutionnel,https://www.theinnovation.eu/innovation-et-cha...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Fiche de lecture, Innovation, Changement inst..."
3,\n \nLe CNC et Oseo ont pour la première fois...,"[\n, [ , [], , []], \n, [Le CNC et Oseo ont p...",2017-10-30T10:14:43+00:00,fr_FR,Appel à projet Oseo-CNC pour l’innovation en A...,https://www.theinnovation.eu/appel-a-projet-os...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Financement, Industrie créative, Audio-Visuel..."
4,"\nAvec l’émergence du Web 2.0, on assiste à un...","[\n, [[Avec l’émergence du Web 2.0, on assiste...",2017-10-30T10:14:30+00:00,fr_FR,Les licences du web 2.0,https://www.theinnovation.eu/les-licences-du-w...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Concept de base, Innovation, creative commons..."
5,\nUn nouveau dispositif Oseo-CNC de financemen...,"[\n, [[Un nouveau dispositif Oseo-CNC], de fi...",2017-10-30T10:13:35+00:00,fr_FR,L’appel à projet oseo-cnc : le financement du ...,https://www.theinnovation.eu/appel-a-projet-os...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Etienne Guerry,"[Financement, Innovation, exploitation, Explor..."
...,...,...,...,...,...,...,...,...,...,...,...,...
84,"\nDes livres sur l’innovation, on en trouve de...","[\n, [[Des livres sur l’innovation, on en trou...",2020-01-06T07:56:45+00:00,fr_FR,Un nouveau livre sur l’innovation chez EMS : L...,https://www.theinnovation.eu/un-nouveau-livre-...,TheInnovation,xpath_source,https://www.theinnovation.eu/,http://www.theinnovation.eu/innov/wp-content/u...,Guy Parmentier,"[Fiche de lecture, Innovation, Pédagogie, Inno..."
85,\n \n\n\nLa créativité est la génération d’...,"[\n, [[], , [], , [], , [], , []], [\n, [<...",2020-05-08T09:59:04+00:00,fr_FR,La créativité en temps de crise du Covid 19,https://www.theinnovation.eu/la-creativite-en-...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://www.theinnovation.eu/innov/wp-content/...,Guy Parmentier,[Non classé]
86,"\n «Être seuls ensemble», depuis leurs conf...","[\n, [[], , [], , [], , [], , []], [[<a hr...",2020-05-08T09:59:37+00:00,fr_FR,Les communautés virtuelles de crises,https://www.theinnovation.eu/les-communautes-v...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://www.theinnovation.eu/innov/wp-content/...,Guy Parmentier,"[Analyse, Créativité, Innovation]"
87,\n La presse régionale souffre des conséque...,"[\n, [[], , [], , [], , [], , []], [[<img ...",2020-05-14T15:17:09+00:00,fr_FR,Sauver la presse régionale : les leçons de la ...,https://www.theinnovation.eu/sauver-la-presse-...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://www.theinnovation.eu/innov/wp-content/...,Guy Parmentier,"[Analyse, Business Model]"


In [85]:
url = "https://www.theinnovation.eu/comment-tuer-linnovation-avec-lanalyse-financiere/45"

In [86]:
scrap_all(url)

https://www.theinnovation.eu/fichier-edwige-innovation-incrementale-innovation-radicale-ou-innovation-de-rupture/44
https://www.theinnovation.eu/linnovation-avec-les-lead-user/42
https://www.theinnovation.eu/comment-eviter-les-catastrophes-ecologiques/41
https://www.theinnovation.eu/construction-identitaire-dans-les-mondes-virtuels/39
https://www.theinnovation.eu/innover-sur-le-business-model/36
https://www.theinnovation.eu/la-generation-didees-dutilisateurs-de-services-bases-sur-de-la-technologie/33
https://www.theinnovation.eu/les-business-model-de-lopen-source/29
https://www.theinnovation.eu/management-des-idees-et-open-innovation/28
https://www.theinnovation.eu/appel-a-projet-oseo-cnc-une-tendance-au-developpement-plutot-qu%e2%80%99a-la-recherche/26
https://www.theinnovation.eu/les-licences-du-web-20/20
https://www.theinnovation.eu/appel-a-projet-oseo-cnc-pour-linnovation-en-audio-visuel-et-multimedia/14
https://www.theinnovation.eu/innovation-et-changement-institutionnel/11
https:

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
1,\n\nOn ne parle pas souvent d’innovation dans ...,"[\n, [[<img alt="""" src=""http://www.theinnovati...",2010-05-14T16:05:07+00:00,fr_FR,29-30 novembre – Forum Innovation et Tourisme,https://www.theinnovation.eu/le-forum-innovati...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Agenda, Innovation, Second Life, Tourisme]"
2,\n\nOn ne parle pas souvent d’innovation dans ...,"[\n, [[<img alt="""" src=""http://www.theinnovati...",2010-05-14T16:05:07+00:00,fr_FR,29-30 novembre – Forum Innovation et Tourisme,https://www.theinnovation.eu/le-forum-innovati...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Agenda, Innovation, Second Life, Tourisme]"
3,\n\nOn ne parle pas souvent d’innovation dans ...,"[\n, [[<img alt="""" src=""http://www.theinnovati...",2010-05-14T16:05:07+00:00,fr_FR,29-30 novembre – Forum Innovation et Tourisme,https://www.theinnovation.eu/le-forum-innovati...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Agenda, Innovation, Second Life, Tourisme]"
4,\n\nOn ne parle pas souvent d’innovation dans ...,"[\n, [[<img alt="""" src=""http://www.theinnovati...",2010-05-14T16:05:07+00:00,fr_FR,29-30 novembre – Forum Innovation et Tourisme,https://www.theinnovation.eu/le-forum-innovati...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Agenda, Innovation, Second Life, Tourisme]"
5,\n\nOn ne parle pas souvent d’innovation dans ...,"[\n, [[<img alt="""" src=""http://www.theinnovati...",2010-05-14T16:05:07+00:00,fr_FR,29-30 novembre – Forum Innovation et Tourisme,https://www.theinnovation.eu/le-forum-innovati...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Agenda, Innovation, Second Life, Tourisme]"
...,...,...,...,...,...,...,...,...,...,...,...,...
84,\n\nOn ne parle pas souvent d’innovation dans ...,"[\n, [[<img alt="""" src=""http://www.theinnovati...",2010-05-14T16:05:07+00:00,fr_FR,29-30 novembre – Forum Innovation et Tourisme,https://www.theinnovation.eu/le-forum-innovati...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Agenda, Innovation, Second Life, Tourisme]"
85,\n\nOn ne parle pas souvent d’innovation dans ...,"[\n, [[<img alt="""" src=""http://www.theinnovati...",2010-05-14T16:05:07+00:00,fr_FR,29-30 novembre – Forum Innovation et Tourisme,https://www.theinnovation.eu/le-forum-innovati...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Agenda, Innovation, Second Life, Tourisme]"
86,\n\nOn ne parle pas souvent d’innovation dans ...,"[\n, [[<img alt="""" src=""http://www.theinnovati...",2010-05-14T16:05:07+00:00,fr_FR,29-30 novembre – Forum Innovation et Tourisme,https://www.theinnovation.eu/le-forum-innovati...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Agenda, Innovation, Second Life, Tourisme]"
87,\n\nOn ne parle pas souvent d’innovation dans ...,"[\n, [[<img alt="""" src=""http://www.theinnovati...",2010-05-14T16:05:07+00:00,fr_FR,29-30 novembre – Forum Innovation et Tourisme,https://www.theinnovation.eu/le-forum-innovati...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[Agenda, Innovation, Second Life, Tourisme]"
